In [1]:
# Import Packages:
from __future__ import print_function

import os, sys
from six.moves import cPickle as pickle
from six.moves import range

# Model Packeges import
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix

# Get the current working directory
cwd = os.getcwd()
mydir = os.path.abspath(os.path.join(cwd, ".."))
sys.path.append(mydir)
from DataGenerator import genTrainValidFolds

In [2]:
STDbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/STD/batchData/"
EDGbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/EDG/batchData/"
HOGp1batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp1/batchData/"   # dim =162
HOGp2batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp2/batchData/"   # dim =576
HOGp3batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp3/batchData/"

featureDIR = STDbatch_dir

In [3]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
    
def reshape_data(dataset, labels=None, imageSize=32, numChannels=1, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape((-1,imageSize,imageSize,numChannels)) # To reshape the
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        if labels:
            numLabels = len(np.unique(labels))
            labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape((-1,imageSize,imageSize,numChannels)) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        if labels:
            numLabels = len(np.unique(labels))
            labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [ ]:
class GlabalVariable():
    def __init__():
        

In [8]:
class BuildConvNet():
    
    def __init__(self):
#         self.batchSize = 128
        self.imageSize = 32
        self.numLabels = 2
        self.numChannel = 1              # numChannels -> For grayscale =1, for RGB =3
        self.numHidden = 1000            # numHiddenUnits for the fully connected layerr
#         self.numHidden2 = 1000            # numHiddenUnits for the fully connected layer

        self.conv1Kernel = 5             # Size of kernel for the first convolution layer
        self.conv2Kernel = 5             # Size of kernel for the second convolution layer
        self.conv1Depth = 32             # Number of kernels for the first convolution layer
        self.conv2Depth = 64             # Number of kernels for the second convolution layer
        self.conv1Stride = 1             # Strides for the first convolution layer filter  
        self.conv2Stride = 1             # Strides for the second convolution layer filter

        self.pool1Kernel = 2             # Size of kernel for the first Pooling layer
        self.pool2Kernel = 2             # Size of kernel for the second Pooling layer  
        self.pool1Stride = 2             # Strides for the first Pooling layer filter
        self.pool2Stride = 2             # Strides for the second Pooling layer filter
        
        self.alpha = 0.005               # The learning Rate, A high Learning Rate will would be inefficient. If using higher
                                         # learning rate then use regularizer else, the model will overfitt
        self.dropout = 0.75              # Drop out a weight with a probability of 0.5
        
        self.m = 0.9
        
        self.seed = 2316                 # Set the seeds for the random weights generator
        
        self.weights = {
            'cv1_wght': tf.Variable(tf.random_normal([self.conv1Kernel, self.conv1Kernel, self.numChannel, self.conv1Depth], seed=self.seed)),
            'cv2_wgth': tf.Variable(tf.random_normal([self.conv2Kernel, self.conv2Kernel, self.conv1Depth, self.conv2Depth], seed=self.seed)),
            'out_wght': tf.Variable(tf.random_normal([self.numHidden, self.numLabels], seed=self.seed))
        }

        self.biases = {
            'cv1_bias': tf.Variable(tf.random_normal([self.conv1Depth], seed=self.seed)),
            'cv2_bias': tf.Variable(tf.random_normal([self.conv2Depth], seed=self.seed)),
            'fc1_bias': tf.Variable(tf.random_normal([self.numHidden], seed=self.seed)),
            'out_bias': tf.Variable(tf.random_normal([self.numLabels], seed=self.seed))
        }




    def convLayer(self, x, w, b, s=1, nlModel='RELU'):
        x = tf.nn.conv2d(x, w, [1,s,s,1], padding='SAME') # Same padding
        x = tf.nn.bias_add(x, b)
        if nlModel == 'RELU':
            return tf.nn.relu(x)
        elif nlModel == 'LOGIT':
            return tf.sigmoid(x)


    def poolLayer(self, x, k=2, s=2, poolType='MAX'):
        if poolType=='MAX':
            return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,s,s,1], padding='SAME')
        elif poolType == 'AVG':
            return tf.nn.avg_pool(data, ksize=[1,k,k,1], strides=[1,s,s,1], padding='SAME')

        
    def fcLayer(self, x, w, b, nModel='RELU'):
        x = tf.matmul(x, w) + b
        if nModel=='RELU':
            return tf.nn.relu(x)
        if nModel=='LOGIT':
            return tf.sigmoid(x)
        
        
    def outputLayer(self, x, w, b):
        x = tf.matmul(x, w) + b
        return x, tf.nn.softmax(x)
            
                         
    # Create the entire Model
    def trainGraph(self):
        trainData = tf.placeholder(tf.float32, [None, self.imageSize, self.imageSize, self.numChannel])
        trainLabels = tf.placeholder(tf.float32, [None, self.numLabels])


        # Convolution Layer 1
        conv1 = self.convLayer(trainData, self.weights['cv1_wght'], self.biases['cv1_bias'], s=self.conv1Stride)
        conv1 = self.poolLayer(conv1, k=self.pool1Kernel)

        # Convolution Layer 2
        conv2 = self.convLayer(conv1, self.weights['cv2_wgth'], self.biases['cv2_bias'], self.conv2Stride)
        conv2 = self.poolLayer(conv2, k=self.pool2Kernel)
        poolSize = conv2.get_shape().as_list()[1]

        # Fully connected layer : Reshape conv2 output to fit fully connected layer input
        flattenedPoolSize = poolSize*poolSize*self.conv2Depth
        self.weights['fc1_wght'] =  tf.Variable(tf.random_normal([flattenedPoolSize, self.numHidden]))
        fc1State = tf.reshape(conv2, [-1, flattenedPoolSize])
        fc1State = self.fcLayer(fc1State, self.weights['fc1_wght'], self.biases['fc1_bias'])
        
        # Drop out regularizer
        fc1State = tf.nn.dropout(fc1State, self.dropout)


        # Output, class prediction
        pred, outputState = self.outputLayer(fc1State, self.weights['out_wght'], self.biases['out_bias'])
        lossCE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=trainLabels))
        optimizer = tf.train.AdamOptimizer(learning_rate=self.alpha).minimize(lossCE)
#         optimizer = tf.train.MomentumOptimizer(self.alpha, 
#                                             self.m, 
#                                             use_locking=False, 
#                                             name='Momentum', 
#                                             use_nesterov=False).minimize(lossCE)

        # Evaluate model
#         correct_pred = tf.equal(tf.argmax(outputState, 1), tf.argmax(trainLabels, 1))
#         accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

        
        return dict(
            trainData=trainData, 
            trainLabels=trainLabels, 
            optimizer=optimizer, 
            lossCE=lossCE,
            trainPred = outputState
#             accuracy=accuracy
        )
    
    
    def validGraph(self):

        validData = tf.placeholder(tf.float32, [None, self.imageSize, self.imageSize, self.numChannel])
        validLabels = tf.placeholder(tf.float32, [None, self.numLabels])

        # Convolution Layer 1
        vConv1 = self.convLayer(validData, self.weights['cv1_wght'], self.biases['cv1_bias'], s=self.conv1Stride)
        vConv1 = self.poolLayer(vConv1, k=self.pool1Kernel)

        # Convolution Layer 2
        vConv2 = self.convLayer(vConv1, self.weights['cv2_wgth'], self.biases['cv2_bias'], self.conv2Stride)
        vConv2 = self.poolLayer(vConv2, k=self.pool2Kernel)
        vPoolSize = vConv2.get_shape().as_list()[1]

        # Fully connected layer : Reshape conv2 output to fit fully connected layer input
        vflattenedPoolSize = vPoolSize*vPoolSize*self.conv2Depth
        vFc1State = tf.reshape(vConv2, [-1, vflattenedPoolSize])
        vFc1State = self.fcLayer(vFc1State, self.weights['fc1_wght'], self.biases['fc1_bias'])

        # Using DropOut Regularizer
#         fc1State = tf.nn.dropout(fc1State, self.dropout)

        # Output, class prediction
        vPred, vOutState = self.outputLayer(vFc1State, self.weights['out_wght'], self.biases['out_bias'])

    #             validPred = tf.equal(tf.argmax(validOutState, 1), tf.argmax(validLabels, 1))
    #             validAccuracy = tf.reduce_mean(tf.cast(validPred, tf.float32))

        return dict(
            validData = validData,
            validLabels = validLabels,
            validPred = vOutState
        )




In [ ]:
def accuracy(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

def confusionMatrix(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (pd.crosstab(np.argmax(labels, 1), np.argmax(predictions, 1), rownames=['True'], colnames=['Predicted'], margins=True))


class SessionExec():
    
    def __init__(self):
        self.epochs = 20
        self.imageSize = 32
        self.batchSize = 500
        
        
    def trainModel(self, trainDataIN, trainLabelsIN, sess):    
        numBatches = int(trainDataIN.shape[0]/self.batchSize)
        for numBatch in np.arange(numBatches):
#                     print ('Batch no: ', numBatch)
#                     print (numBatch*self.batchSize,  (numBatch+1)*self.batchSize)
            batchData = trainDataIN[numBatch*self.batchSize : (numBatch+1)*self.batchSize]
            batchLabels = trainLabelsIN[numBatch*self.batchSize : (numBatch+1)*self.batchSize]

            feed_dict = {self.trainGraphDict['trainData']: batchData,
                         self.trainGraphDict['trainLabels']: batchLabels
#                              self.trainGraphDict['keep_prob']: 0.7
                    }

            _, loss, tpred = sess.run([self.trainGraphDict['optimizer'],
                                        self.trainGraphDict['lossCE'],
                                        self.trainGraphDict['trainPred']], feed_dict=feed_dict)
            
            if ((numBatch+1)%5 == 0) or ((numBatch+1) == numBatches): 
                tacc = accuracy(tpred, batchLabels)
                print ("Fold: " + str(self.foldNUM+1) + 
                       ", Epoch: " + str(self.epoch+1)+ 
                       ", Mini Batch: " + str(numBatch+1) + 
                       ", Loss= " + "{:.6f}".format(loss) + 
                       ", Training Accuracy= " + "{:.5f}".format(tacc))
    

    
    def validModel(self, validDataIN, validLabelsIN, sess):
        feed_dict = {self.validGraphDict['validData']: validDataIN,
                     self.validGraphDict['validLabels']: validLabelsIN
#                              self.trainGraphDict['keep_prob']: 0.7
                }

        vpred = sess.run(self.validGraphDict['validPred'], feed_dict=feed_dict)
        
        vacc = accuracy(vpred, validLabelsIN)
        print ("Fold: " + str(self.foldNUM+1) + 
                ", Epoch: " + str(self.epoch+1)+ 
                ", Validation Accuracy= " + "{:.5f}".format(vacc))
        
        return vpred 
        
        
    def execute(self, updWghtBias=False):
        numLabels = 2
        numFolds = 10
        meanValidAcc = 0
        self.wLRND = {}
        self.bLRND = {}
        
        for foldNUM, (trainDataIN, trainLabelsIN, validDataIN, validLabelsIN, labelDict) in enumerate(genTrainValidFolds(featureDIR, oneHot=True)):   
            print ('')
            print ('##########################################################################################')
            trainDataIN, _ = reshape_data(trainDataIN, imageSize=self.imageSize, numChannels=1)                                       
            validDataIN, _ = reshape_data(validDataIN, imageSize=self.imageSize, numChannels=1)
            
            print ('')
            print ('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
            print ('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
            print ('The Label Dictionary is given as: ', labelDict)
            print ('')
            
            self.foldNUM = foldNUM

            reset_graph()

            # Create a object encapsulating the graph lineage
            objCNN = BuildConvNet()
            self.trainGraphDict = objCNN.trainGraph()
            self.validGraphDict = objCNN.validGraph()
            
            with tf.Session() as sess:
                sess.run(tf.initialize_all_variables())

                for epoch in range(self.epochs): 
                    self.epoch = epoch
                    
                    # Train The model for Epochs
                    self.trainModel(trainDataIN, trainLabelsIN, sess)
                    print ('')
                    
                    # Validate the model for Epochs
                    vpred = self.validModel(validDataIN, validLabelsIN, sess)
                    print ('')

                    validCM = confusionMatrix(validLabelsIN,vpred)
                    print ('Confusion Matrix CrossValid Set')
                    print (validCM)
                    print ('')
                    
            if foldNUM == 2:
                break

SessionExec().execute(updWghtBias=False)   


##########################################################################################

Validation Data and Labels shape:  (1000, 32, 32, 1) (1000, 2)
Training Data and Labels shape:  (9000, 32, 32, 1) (9000, 2)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}

Fold: 1, Epoch: 1, Mini Batch: 5, Loss= 34088.933594, Training Accuracy= 55.20000
Fold: 1, Epoch: 1, Mini Batch: 10, Loss= 17000.271484, Training Accuracy= 66.00000
Fold: 1, Epoch: 1, Mini Batch: 15, Loss= 13483.639648, Training Accuracy= 68.80000
Fold: 1, Epoch: 1, Mini Batch: 18, Loss= 12162.555664, Training Accuracy= 66.80000

Fold: 1, Epoch: 1, Validation Accuracy= 64.40000

Confusion Matrix CrossValid Set
Predicted    0    1   All
True                     
0          151    7   158
1          349  493   842
All        500  500  1000

Fold: 1, Epoch: 2, Mini Batch: 5, Loss= 5706.247559, Training Accuracy= 71.40000
Fold: 1, Epoch: 2, Mini Batch: 10, Loss= 4824.378906, Training 